* changed by nov05 on 2024-11-24  
* Udacity content: [exercise](https://www.evernote.com/shard/s139/u/0/sh/d22b9fe5-9992-4dd0-9402-c623cdbc90b4/rJcBRGAXxAQfdkl3kqgZ1N2VIKFVDSOLDimPDJFMhwHEQmyRu0AHwQTqxw), [solution](https://www.evernote.com/shard/s139/u/0/sh/00654bdd-0c00-4525-a0e4-beeeccb17e18/UdvEWior2s6PYMczVqGFVPf1PM_g35bFRrPiVlm79SfmnVhyPY_BJQPsiw)    
* my notes:
  * [Feature Store](https://docs.google.com/document/d/1pZtRTDmZYiaHO4kFcGYEDSneVZ98klFH8E4YR2hPUAk)  
  * [Data quanlity and Clarify model monitors](https://docs.google.com/document/d/1ICJvBqJqBtDuTwTkyB10egC20sBSqFtdWq7zk0BEuQA)   

* Tips: 
  * I was able to run this notebook on my local computer to train and deploy model, create monitors. 
  * I would have been able to create Feature Store group as well if the assumed role had the permissions. 
  * To run SageMaker in the local environment, add the following 3 keys to `~/.aws/credentials`.
    ```bash  
    (awsmle_py310) PS D:\github\udacity-nd009t-C2-Developing-ML-Workflow> cat ~/.aws/credentials
    [default]
    aws_access_key_id = *KDV4
    aws_secret_access_key = *U4jB
    aws_session_token = *vQ==    
    ```

# Exercises

This is the notebook containing the exercises for **Feature Store**, **Model Monitor**, and **Clarify**. Tested for these exercises was performed using __2 vCPU + 4 GiB notebook instance with Python 3 (TensorFlow 2.1 Python 3.6 CPU Optimized) kernel__.

## Staging

We'll begin by initializing some variables. These are often assumed to be present in code samples you'll find in the AWS documenation.

In [1]:
import sagemaker # type: ignore
from sagemaker import get_execution_role # type: ignore
from sagemaker.session import Session # type: ignore

role_arn = get_execution_role()  ## get role ARN
if 'AmazonSageMaker-ExecutionRole' not in role_arn:
    role_arn = "arn:aws:iam::807711953667:role/service-role/AmazonSageMaker-ExecutionRole-20241121T213663"
print("Role ARN:", role_arn) ## If local, Role ARN: arn:aws:iam::807711953667:role/voclabs
session = sagemaker.Session()
region = session.boto_region_name
# bucket = session.default_bucket()
bucket = "sagemaker-studio-807711953667-mmx0am1bt28"

d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[11/24/24 18:11:30] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=883390;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=919137;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\guido\AppData\Local\sagemaker\sagemaker\config.yaml


[11/24/24 18:11:32] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=932782;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=318265;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[11/24/24 18:11:34] WARNING  Couldn't call 'get_role' to get Role ARN from role name voclabs to get ]8;id=338138;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=461607;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#5902\5902]8;;\
                             Role path.                                                                            

Role ARN: arn:aws:iam::807711953667:role/service-role/AmazonSageMaker-ExecutionRole-20241121T213663


                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=439693;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=390344;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

## **👉 Feature Store**  
---

* Check [the Google Docs notes](https://docs.google.com/document/d/1pZtRTDmZYiaHO4kFcGYEDSneVZ98klFH8E4YR2hPUAk)   

Feature Store is a special database to give ML systems a consistent data flow across training and inference workloads. It can ingest data in batches (for training) as well as serve input features to models with very low latency for real-time prediction.

For this exercise we'll work with a wine quality dataset: https://archive.ics.uci.edu/ml/datasets/wine+quality/

```
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.
```

In [2]:
import pandas as pd # type: ignore
from sklearn import datasets # type: ignore
import time
# import uuid

data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
print(df.columns)

Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline'],
      dtype='object')


If we leave the column names as-is, Feature Store won't be able to handle the `/` in `od280/od315_of_diluted_wines` (`/` is a delimiter Feature Store uses to manage how features are organized.)

In [3]:
df.rename(columns={'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)
## Add columns for feature group
df["EventTime"] = time.time()
df["ID"] = range(len(df))
df.sample(3)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline,EventTime,ID
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,1.732480e+09,4
154,12.58,1.29,2.10,20.0,103.0,1.48,0.58,0.53,1.40,7.60,0.58,1.55,640.0,1.732480e+09,154
161,13.69,3.26,2.54,20.0,107.0,1.83,0.56,0.50,0.80,5.88,0.96,1.82,680.0,1.732480e+09,161


Once we have our data, we can create a feature group. Remember to attach event time and ID columns - Feature Store needs them.

In [5]:
from sagemaker.feature_store.feature_group import FeatureGroup # type: ignore
 
# TODO: Create feature group
feature_group_name = "wine-features"
feature_group = FeatureGroup(
    name=feature_group_name, 
    sagemaker_session=session
)
# TODO: Load Feature definitions
feature_group.load_feature_definitions(data_frame=df)

[FeatureDefinition(feature_name='alcohol', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='malic_acid', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='alcalinity_of_ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='magnesium', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='total_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='flavanoids', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='nonflavanoid_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collec

The feature group is not created until we call the `create` method, let's do that now:

In [6]:
# Create the feature store:
feature_group.create(
    s3_uri=f"s3://{bucket}/features",
    record_identifier_name='ID',
    event_time_feature_name="EventTime",
    role_arn=role_arn,
    enable_online_store=True,
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:807711953667:feature-group/wine-features',
 'ResponseMetadata': {'RequestId': '9adc9df7-1db7-4c01-85ea-063dc7aa4f9d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9adc9df7-1db7-4c01-85ea-063dc7aa4f9d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '90',
   'date': 'Sun, 24 Nov 2024 18:28:03 GMT'},
  'RetryAttempts': 0}}

🟢⚠️ Issue solved: I got the following response when creating from a local notebook. However, the creation failed. Go to `SageMaker Studio > Data > Feature Store`. Click on the feature group. Click on the “Details” tab. It seems the assumed role doesn’t have certain permissions. While creating from a SageMaker notebook, it succeeded. Data is written in Parquet format with partitions to AWS S3.      
```
{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:807711953667:feature-group/wine-features',
 'ResponseMetadata': {'RequestId': 'd49e3486-e1cb-414e-86fe-0f56d5fbf5fa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd49e3486-e1cb-414e-86fe-0f56d5fbf5fa',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '90',
   'date': 'Sun, 24 Nov 2024 17:56:18 GMT'},
  'RetryAttempts': 0}}
```

In [7]:
feature_group = FeatureGroup(name=feature_group_name)
feature_group_status = feature_group.describe()['FeatureGroupStatus']
print(feature_group_status, type(feature_group_status))  ## CreateFailed <class 'str'>

Created <class 'str'>


Lastly, ingest some data into your feature group:

In [8]:
# TODO
if feature_group_status=='Created':
    feature_group.ingest(data_frame=df, 
                        max_workers=5, 
                        wait=True)
else:
    print('⚠️ The feature group is not created.')

Great job! You've demonstrated your understanding of creating feature groups and ingesting data into them using Feature Store. Next up we'll cover Model Monitor!

## **👉 Model Monitor**    

---   

* Check [the Google Docs notes](https://docs.google.com/document/d/1ICJvBqJqBtDuTwTkyB10egC20sBSqFtdWq7zk0BEuQA)    

In this exercise we'll create a monitoring schedule for a deployed model. We're going to provide code to help you deploy a model and get started, so that you can focus on Model Monitor for this exercise. __Remember to clean up your model before you end a work session__. We'll provide some code at the end to help you clean up your model. We'll begin by reloading our data from the previous exercise.



In [2]:
import pandas as pd # type: ignore
from sklearn import datasets # type: ignore

data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html  

* Tips:   
  * Ensuring the 'TARGET' Column is the First Column:  
  *After the df.reset_index(inplace=True), the 'TARGET' column becomes the first column in the DataFrame. If the intention was to reorder the DataFrame columns, this sequence achieves that by popping the 'TARGET' column, making it the index, and then converting it back to a regular column (now at the front).*
  * Removing the Column Before Resetting It:  
  *df.pop('TARGET') removes 'TARGET' from the DataFrame, ensuring it is not duplicated when resetting the index (as reset_index() would otherwise add the index back as a new column).*

In [3]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)
df.sample(3)

,TARGET,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280_od315_of_diluted_wines,proline
14,0,14.38,1.87,2.38,12.0,102.0,3.30,3.64,0.29,2.96,7.50,1.20,3.00,1547.0
157,2,12.45,3.03,2.64,27.0,97.0,1.90,0.58,0.63,1.14,7.50,0.67,1.73,880.0
61,1,12.64,1.36,2.02,16.8,100.0,2.02,1.41,0.53,0.62,5.75,0.98,1.59,450.0


Now we'll upload the data to S3 as train and validation data:

In [ ]:
import boto3
from io import BytesIO #StringIO

s3_client = boto3.client('s3')
delimiter = int(len(df)/2)
df_train, df_val = df.iloc[delimiter:], df.iloc[:delimiter]

## prepare training data 
csv_buffer = BytesIO()
df_train.to_csv(csv_buffer, header=False, index=False)  ## send to buffer
csv_buffer.seek(0)
s3_key = "wine/data/train.csv"  
s3_client.put_object(Body=csv_buffer, Bucket=bucket, Key=s3_key)  ## uplaod to S3
train_input = sagemaker.inputs.TrainingInput(
    s3_data=f"s3://{bucket}/{s3_key}", 
    content_type='csv')
print(f"s3://{bucket}/{s3_key}") 

## prepare validation data
csv_buffer = BytesIO()
df_val.to_csv(csv_buffer, header=False, index=False)  ## send to buffer
csv_buffer.seek(0)
s3_key = "wine/data/validation.csv" 
s3_client.put_object(Body=csv_buffer, Bucket=bucket, Key=s3_key)  ## upload to S3
val_input = sagemaker.inputs.TrainingInput(
    s3_data=f"s3://{bucket}/{s3_key}", 
    content_type='csv')
print(f"s3://{bucket}/{s3_key}")

s3://sagemaker-studio-807711953667-mmx0am1bt28/wine/data/train.csv
s3://sagemaker-studio-807711953667-mmx0am1bt28/wine/data/validation.csv


In [ ]:
%%time
algo_image = sagemaker.image_uris.retrieve("xgboost", region, version='latest')
s3_model_output = f"s3://{bucket}/wine/model"

model=sagemaker.estimator.Estimator(
    image_uri=algo_image,
    role=role_arn,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_model_output,
    sagemaker_session=session
)
model.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    objective='reg:linear',
    early_stopping_rounds=10,
    num_round=200
)
model.fit({
    'train': train_input, 
    'validation': val_input
})
## go to "SageMaker - Training - Training jobs". Make sure the job is completed.
## CPU times: total: 4.05 s
## Wall time: 3min 33s

[11/24/24 19:39:20] INFO     Ignoring unnecessary instance type: None.                            ]8;id=13996;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=387960;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#523\523]8;;\

                    INFO     Creating training-job with name: xgboost-2024-11-25-01-39-20-779       ]8;id=674815;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=316261;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#1042\1042]8;;\

2024-11-25 01:39:27 Starting - Starting the training job...
2024-11-25 01:39:50 Starting - Preparing the instances for training......
2024-11-25 01:40:45 Downloading - Downloading input data...
2024-11-25 01:41:10 Downloading - Downloading the training image.....Arguments: train
[2024-11-25:01:42:27:INFO] Running standalone xgboost training.
[2024-11-25:01:42:27:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 8456.52mb
[2024-11-25:01:42:27:INFO] Determined delimiter of CSV input is ','
[01:42:27] S3DistributionType set as FullyReplicated
[01:42:27] 89x13 matrix with 1157 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-11-25:01:42:27:INFO] Determined delimiter of CSV input is ','
[01:42:27] S3DistributionType set as FullyReplicated
[01:42:27] 89x13 matrix with 1157 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[01:42:27] src/tree/updater_prune.cc:74: tree pruni

Now that your training job has finished, you can perform the first task in this exercise:   
* Creating a data capture config. Configure your model to sample `34%` of inferences.  

In [23]:
%%time
# TODO
from sagemaker.model_monitor import DataCaptureConfig # type: ignore

destination_s3_uri = f's3://{bucket}/wine/data-capture'
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=34,
    destination_s3_uri=destination_s3_uri
)

CPU times: total: 46.9 ms
Wall time: 83.1 ms


Great! We'll use your config to deploy a model below:

In [24]:
%%time
xgb_predictor = model.deploy(
    initial_instance_count=1, 
    instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)
## go to "SageMaker - Inference - Endpoints" to check the result.

[11/24/24 19:48:49] INFO     Creating model with name: xgboost-2024-11-25-01-48-48-543              ]8;id=941270;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=366574;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#4025\4025]8;;\

                    INFO     Creating endpoint-config with name xgboost-2024-11-25-01-48-48-543     ]8;id=713305;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=580388;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#5820\5820]8;;\

[11/24/24 19:48:50] INFO     Creating endpoint with name xgboost-2024-11-25-01-48-48-543            ]8;id=404575;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=458316;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#4642\4642]8;;\

------!

Great! You should see an indicator like this when the deployment finishes:

```
-----------------!
```
We can test your deployment like so:

In [ ]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
x_pred = xgb_predictor.predict(
    df_val.drop(columns=df_val.columns[0]).sample(5).values  ## Drop the target column
).decode('utf-8')
x_pred

'0.6030303239822388,0.6030303239822388,0.6030303239822388,0.6030303239822388,0.6030303239822388'

All systems go! To finish up the exercise, we're going to provide you with a `DefaultModelMonitor` and a suggested baseline. Combine the `xgb_predictor` and the provided `my_monitor` to configure the monitoring schedule for _hourly_ monitoring.

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor # type: ignore
from sagemaker.model_monitor.dataset_format import DatasetFormat # type: ignore

my_monitor = DefaultModelMonitor(
    role=role_arn,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)
## Check the baseline job in "Amazon SageMaker > Processing jobs"

[11/24/24 20:00:29] INFO     Defaulting to the only supported framework/algorithm version: .      ]8;id=733528;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=119760;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#487\487]8;;\

[11/24/24 20:00:30] INFO     Ignoring unnecessary instance type: None.                            ]8;id=241663;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=80900;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#523\523]8;;\

In [ ]:
%%time
my_monitor.suggest_baseline(
    baseline_dataset=f"s3://{bucket}/wine/data/train.csv",
    dataset_format=DatasetFormat.csv(header=False),
)
## 5m 24.3s

[11/24/24 20:00:41] INFO     Creating processing-job with name                                      ]8;id=836295;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=184596;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#1575\1575]8;;\
                             baseline-suggestion-job-2024-11-25-02-00-38-911                                       

...........2024-11-25 02:02:44.806930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-25 02:02:44.806963: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-11-25 02:02:46.537449: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-11-25 02:02:46.537481: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-11-25 02:02:46.537504: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-238-171.ec2.internal): /proc/driver/nvidia/version does not exist
2024-11-25 02:02:46.537782: I tensorf

Below, provide the monitoring schedule:

In [ ]:
# TODO
from sagemaker.model_monitor import CronExpressionGenerator # type: ignore

my_monitor.create_monitoring_schedule(
    monitor_schedule_name='wine-monitoring-schedule',
    endpoint_input=xgb_predictor.endpoint_name,
    statistics=my_monitor.baseline_statistics(),
    constraints=my_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)
## Go to "Amazon SageMaker > Model dashboard > <your model> > Monitor schedule" to check the result.

[11/24/24 20:07:32] INFO     Creating Monitoring Schedule with name:                       ]8;id=159074;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\model_monitoring.py\model_monitoring.py]8;;\:]8;id=963093;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\model_monitoring.py#1560\1560]8;;\
                             wine-monitoring-schedule                                                              

Great job! You can check that your schedule was created by ~~selecting the `SageMaker components and registries` tab on the far left~~. In this exercise you configured Model Monitor to watch a simple model. Next, we'll monitor the same deployment for explainability.

* Go to `Amazon SageMaker > Model dashboard > <your model> > Monitor schedule`   

⚠️ __REMINDER:__ Don't leave your model deployed overnight. If you aren't going to follow up with the Clarify exercise within a few hours, use the code below to remove your model:

In [ ]:
## delete the monitors and endpoint
# monitors = xgb_predictor.list_monitors()
# for monitor in monitors:
#     monitor.delete_monitoring_schedule()
# xgb_predictor.delete_endpoint()

## **👉 Clarify**  

For the last exercise we'll deploy an explainability monitor using [`Clarify`](https://aws.amazon.com/sagemaker/clarify/). We're going to use the model that you deployed in the last exercise, but if you cleaned up your deployments from the previous exercise, that's ok! You can rerun the deployment from the previous exercise up to the point where we deployed our model. It'll look like this:

```python
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)
```

Once your model is deployed, you can come back here. _REMINDER_: you need to clean up your deployment, don't leave it running overnight. We'll provide some code at the end to delete your deployment.

*  Amazon SageMaker Examples:    
  [Fairness and Explainability with `SageMaker Clarify`](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-clarify/fairness_and_explainability/fairness_and_explainability.html)   

## Prep

We'll begin by reloading our data from the previous exercise.

In [ ]:
# import pandas as pd # type: ignore
# from sklearn import datasets # type: ignore
# data = datasets.load_wine()
# df = pd.DataFrame(data['data'])
# df.columns = data['feature_names']
# df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [ ]:
# df["TARGET"] = data['target']
# df.set_index(df.pop('TARGET'), inplace=True)
# df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [ ]:
# import boto3
# from io import BytesIO #StringIO

# s3_client = boto3.client('s3')
# delimiter = int(len(df)/2)
# df_train, df_val = df.iloc[delimiter:], df.iloc[:delimiter]

# ## prepare training data 
# csv_buffer = BytesIO()
# df_train.to_csv(csv_buffer, header=False, index=False)  ## send to buffer
# csv_buffer.seek(0)
# s3_key = "wine/data/train.csv"  
# s3_client.put_object(Body=csv_buffer, Bucket=bucket, Key=s3_key)  ## uplaod to S3
# train_input = sagemaker.inputs.TrainingInput(
#     s3_data=f"s3://{bucket}/{s3_key}", 
#     content_type='csv')
# print(f"s3://{bucket}/{s3_key}") 

# ## prepare validation data
# csv_buffer = BytesIO()
# df_val.to_csv(csv_buffer, header=False, index=False)  ## send to buffer
# csv_buffer.seek(0)
# s3_key = "wine/data/validation.csv" 
# s3_client.put_object(Body=csv_buffer, Bucket=bucket, Key=s3_key)  ## upload to S3
# val_input = sagemaker.inputs.TrainingInput(
#     s3_data=f"s3://{bucket}/{s3_key}", 
#     content_type='csv')
# print(f"s3://{bucket}/{s3_key}")

Great! Our data is staged and our model is deployed - let's monitor it for explainability. We need to define three config objects, the `SHAPConfig`, the `ModelConfig`, and the `ExplainabilityAnalysisConfig`. Below, we provide the `SHAPConfig`.

In [31]:
shap_config = sagemaker.clarify.SHAPConfig(
    baseline=[df_train.mean().astype(int).to_list()[1:]],
    num_samples=int(df_train.size),
    agg_method="mean_abs",
    save_local_shap_values=False,
)

Next up, fill in the blanks to define the `ModelConfig` and `ExplainabilityAnalysisConfig`.

In [ ]:
# TODO
model_config = sagemaker.clarify.ModelConfig(
    model_name=xgb_predictor.endpoint_name,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    content_type="text/csv",
    accept_type="text/csv",
)
analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
    explainability_config=shap_config,
    model_config=model_config,
    headers=df_train.columns.to_list()[1:],
)

Before we apply our config, we need to create the monitor object. This is what we'll apply all our config to.

In [35]:
model_explainability_monitor = (
    sagemaker.model_monitor.ModelExplainabilityMonitor(
        role=role_arn,
        sagemaker_session=session,
        max_runtime_in_seconds=1800)
)

[11/24/24 20:24:03] INFO     Defaulting to the only supported framework/algorithm version: 1.0.   ]8;id=714777;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=130886;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#487\487]8;;\

[11/24/24 20:24:04] INFO     Ignoring unnecessary instance type: None.                            ]8;id=62379;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=210769;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#523\523]8;;\

Everything's ready! Below, create a monitoring schedule using the configs we created. Set the schedule to run _daily_.

In [36]:
# TODO 
from sagemaker.model_monitor import CronExpressionGenerator  # type: ignore

explainability_uri = f"s3://{bucket}/wine/model_explainability"
model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=explainability_uri,
    analysis_config=analysis_config,
    endpoint_input=xgb_predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.daily() #.hourly(),
)

[11/24/24 20:24:06] INFO     Uploading analysis config to {s3_uri}.                 ]8;id=236711;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\clarify_model_monitoring.py\clarify_model_monitoring.py]8;;\:]8;id=666900;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\clarify_model_monitoring.py#227\227]8;;\

[11/24/24 20:24:08] INFO     Creating Monitoring Schedule with name:                       ]8;id=223458;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\model_monitoring.py\model_monitoring.py]8;;\:]8;id=373912;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\model_monitoring.py#1560\1560]8;;\
                             monitoring-schedule-2024-11-25-02-24-06-231                                           

Way to go! You can check that your schedule was created by selecting ~~the `SageMaker components and registries` tab on the far left~~. In this exercise you deployed a monitor for explainability to your SageMaker endpoint. This is the last exercise - you'll apply these learnings again in your Project at the end of the course.



⚠️ __REMINDER:__ Don't leave your model deployed overnight. Use the code below to remove your model:

In [37]:
## delete all the monitors and endpoints
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()
xgb_predictor.delete_endpoint()

[11/24/24 20:32:12] INFO     Defaulting to the only supported framework/algorithm version: 1.0.   ]8;id=558324;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=622303;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#487\487]8;;\

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=110601;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=638033;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#523\523]8;;\

[11/24/24 20:32:13] INFO     Defaulting to the only supported framework/algorithm version: .      ]8;id=773670;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=229123;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#487\487]8;;\

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=693648;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=484323;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#523\523]8;;\

                    INFO     Deleting Monitoring Schedule with name:                                ]8;id=877498;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=926090;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#2237\2237]8;;\
                             monitoring-schedule-2024-11-25-02-24-06-231                                           

[11/24/24 20:32:19] INFO     Deleting Model Explainability Job Definition with     ]8;id=150250;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\clarify_model_monitoring.py\clarify_model_monitoring.py]8;;\:]8;id=541800;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\clarify_model_monitoring.py#1289\1289]8;;\
                             name:                                                                                 
                             model-explainability-job-definition-2024-11-25-02-24-                                 
                             06-231                                                                                

[11/24/24 20:32:20] INFO     Deleting Monitoring Schedule with name: wine-monitoring-schedule       ]8;id=169121;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=922755;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#2237\2237]8;;\

[11/24/24 20:32:30] INFO     Deleting Data Quality Job Definition with name:               ]8;id=925298;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\model_monitoring.py\model_monitoring.py]8;;\:]8;id=685787;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model_monitor\model_monitoring.py#2456\2456]8;;\
                             data-quality-job-definition-2024-11-25-02-07-31-428                                   

[11/24/24 20:32:31] INFO     Deleting endpoint configuration with name:                             ]8;id=206393;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=458157;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#4796\4796]8;;\
                             xgboost-2024-11-25-01-48-48-543                                                       

[11/24/24 20:32:32] INFO     Deleting endpoint with name: xgboost-2024-11-25-01-48-48-543           ]8;id=995407;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=372637;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#4786\4786]8;;\